# Simulation template.

In this notebook we produce the results of the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 654 ms, sys: 48.9 ms, total: 702 ms
Wall time: 729 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_100_6561.csv"   
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_100_6561"
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 9.78 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/linneu/ssh_csvs/ssh2/periodic_180_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/linneu/ssh_simulations/ssh2/periodic_180_6561"
random_state = 193


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/linneu/anaconda3/envs/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180980 entries, 0 to 1180979
Columns: 188 entries, id to feat179
dtypes: float64(183), int32(3), object(2)
memory usage: 1.6+ GB
CPU times: user 28.3 s, sys: 2.17 s, total: 30.5 s
Wall time: 31.1 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  180
n_hamiltonians:  6561
n_ts:  2
CPU times: user 650 µs, sys: 50 µs, total: 700 µs
Wall time: 496 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [01:17<2:08:04, 77.62s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [02:34<2:06:10, 77.25s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [2:05:02<00:00, 75.14s/it]

running experiments: 100%|██████████| 100/100 [2:05:02<00:00, 75.02s/it]

CPU times: user 1d 3h 18min 30s, sys: 3min, total: 1d 3h 21min 31s
Wall time: 2h 5min 2s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.018047,0.037394,...,0.029254,0.015857,0.004441,0.027159,-0.041982,-0.021236,0.005203,-0.024027,0.037394,0.018047
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.084825,-0.026222,...,-0.104377,-0.066035,0.055706,-0.051816,0.080410,0.116916,-0.119805,-0.040657,0.026222,-0.084825
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.043223,0.005021,...,0.026354,-0.048356,0.035233,0.067210,-0.054791,-0.024377,0.024692,-0.018139,-0.005021,0.043223
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.009349,-0.031973,...,0.043557,0.051909,-0.042604,-0.009129,0.022361,-0.026100,0.011385,0.036684,-0.031973,-0.009349
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.028827,0.057672,...,0.053523,0.039631,-0.002767,0.038861,-0.065675,-0.039973,0.010875,-0.033361,0.057672,0.028827


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 254 ms, sys: 4 ms, total: 258 ms
Wall time: 257 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
355466,1974,H_-0.8_-0.5_-0.0101248.dat,-0.80,-0.50,-0.010125,0,0,test,-0.072038,-0.074331,...,0.033730,0.058655,-0.022778,0.028966,-0.053539,-0.035041,-0.076242,-0.064150,-0.074331,-0.072038
414081,2300,H_-0.6_-0.4_-0.0130191.dat,-0.60,-0.40,-0.013019,0,0,test,-0.049733,0.009831,...,-0.104868,0.099639,-0.010840,0.079309,0.093536,-0.039436,0.088198,-0.099061,0.009831,-0.049733
267687,1487,H_-1.1_-0.55_-0.00631085.dat,-1.10,-0.55,-0.006311,0,0,train,-0.128269,0.008906,...,0.138035,-0.132247,-0.105938,-0.038285,-0.080112,0.158733,0.158671,-0.046193,0.008906,-0.128269
525286,2918,H_-0.2_-1.9_2.00319.dat,-0.20,-1.90,2.003190,2,1,val,-0.008832,-0.006689,...,-0.030723,-0.090944,0.058266,0.059882,0.006756,0.039492,-0.010297,-0.035543,-0.006689,-0.008832
703727,3909,H_0.4_-0.95_1.00201.dat,0.40,-0.95,1.002009,1,1,val,0.091670,-0.001257,...,-0.089785,-0.006575,-0.002371,0.093912,0.091924,-0.090359,-0.089250,-0.002872,-0.001257,0.091670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224767,1248,H_-1.25_-0.35_-0.00355514.dat,-1.25,-0.35,-0.003555,0,0,val,-0.056233,0.081562,...,0.112894,-0.060886,0.065823,-0.103289,-0.006215,-0.075779,-0.080290,0.004775,-0.081562,0.056233
448065,2489,H_-0.5_0.95_0.0002992.dat,-0.50,0.95,0.000299,0,0,train,0.040314,-0.060272,...,0.011676,-0.051151,-0.058599,0.001720,0.027745,0.060717,0.030744,-0.056952,-0.060272,0.040314
598595,3325,H_0.05_-1.8_1.00267.dat,0.05,-1.80,1.002668,1,1,train,0.021880,-0.030311,...,-0.117420,0.131704,0.055067,0.019019,-0.017307,-0.093007,-0.003999,0.044431,0.030311,-0.021880
483681,2687,H_-0.35_-1.3_1.00451.dat,-0.35,-1.30,1.004510,1,1,val,0.098263,-0.009454,...,-0.072229,0.035068,-0.015178,-0.095579,0.104644,0.083835,-0.091472,0.011519,0.009454,-0.098263


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [2956, 4468, 6492, 4078, 137, 6046, 4259, 3014, 3653, 4351, 3513, 3752, 3010, 386, 4715, 1265, 4207, 318, 3465, 37, 103, 4692, 6495, 3254, 4011, 2774, 3317, 5998, 1521, 1824, 5401, 60, 4936, 3661, 387, 2739, 4425, 4805, 4988, 5346, 5297, 4406, 4576, 3245, 601, 2142, 4253, 1690, 2192, 2767, 3062, 5077, 557, 1127, 4285, 6035, 4240, 6097, 4785, 6549, 1828, 4215, 6326, 5868, 2825, 6262, 380, 5915, 6205, 53, 269, 799, 4857, 143, 4103, 1418, 128, 2780, 3992, 5685, 6206, 1093, 1825, 881, 2443, 2331, 5617, 1812, 2694, 4287, 2446, 375, 4262, 3751, 6193, 5510, 5212, 807, 1211, 1682, 3453, 517, 3534, 2821, 2693, 1207, 1520, 3304, 6301, 3775, 106, 4775, 2221, 6325, 542, 5067, 2436, 4036, 5456, 6111, 423, 548, 3614, 3472, 1493, 5955,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 291 µs, sys: 1e+03 ns, total: 292 µs
Wall time: 176 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.283333,0.377778,0.166667,0.172222,999,0
1,-2.0,-1.95,test,0.283333,0.322222,0.244444,0.150000,-1,0
2,-2.0,-1.90,test,0.294444,0.311111,0.255556,0.138889,-1,0
3,-2.0,-1.85,test,0.333333,0.283333,0.250000,0.133333,-1,-1
4,-2.0,-1.80,test,0.366667,0.294444,0.244444,0.094444,-1,-1
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.483333,0.222222,0.066667,0.227778,-1,-1
6557,2.0,1.85,test,0.466667,0.211111,0.055556,0.266667,-1,-1
6558,2.0,1.90,test,0.372222,0.255556,0.100000,0.272222,-1,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1180980


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,-1,test
2,0,999,1,test
3,0,999,2,test
4,0,999,-1,test
...,...,...,...,...
1180975,6560,999,-1,test
1180976,6560,999,0,test
1180977,6560,999,0,test
1180978,6560,999,-1,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9997585415912109,
 'eigenvector_val': 0.9596477455716587,
 'eigenvector_test': 0.6453240201785021,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.994927536231884,
 'hamiltonian_test': 0.8940628637951106}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}